# EDA
I have done EDA and Baseline Model[ here](https://www.kaggle.com/arunkumar13111/vehicle-insurance-prediction-auc-84). In this notebook, i will  explore following classification algoithms for understanding.
1. Logistic Regression
2. RandomForest

# Feature Engineering

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.model_selection import StratifiedKFold

In [2]:
# read data 
train = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/train.csv")
test  = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/test.csv")

In [3]:
print(train.info(),test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127037 entries, 0 to 127036
Data columns (total 

In [4]:
#split numerical and categorical feature
num_feature = ["Age","Vintage","Annual_Premium"]
cat_feature = ["Gender","Driving_License","Region_Code","Previously_Insured","Vehicle_Age","Vehicle_Damage","Policy_Sales_Channel"]

# convert into integer
train["Policy_Sales_Channel"] = train["Policy_Sales_Channel"].astype("int")
train["Region_Code"] = train["Region_Code"].astype("int")

In [5]:
psc_tot = train["Policy_Sales_Channel"].value_counts()
train["psc_count"] = train["Policy_Sales_Channel"].map(psc_tot)
psc_scount = train[train["Response"]==1].groupby("Policy_Sales_Channel")["Response"].count()
train["psc_scount"] = train["Policy_Sales_Channel"].map(psc_scount)
train["psc_success_rate"] = (train["psc_scount"]/train["psc_count"])*100
train["psc_success_rate"].fillna(0,inplace=True)
reg_tot = train["Region_Code"].value_counts()
train["reg_count"] = train["Region_Code"].map(reg_tot)
psc_scount = train[train["Response"]==1].groupby("Region_Code")["Response"].count()
train["reg_scount"] = train["Region_Code"].map(psc_scount)
train["reg_success_rate"] = (train["reg_scount"]/train["reg_count"])*100


# Binning

In [6]:
#Binning the Age 
train["Age_Cat"]= pd.cut(train["Age"],bins=[10,20,30,40,50,60,70,80,90,100],labels=[1,2,3,4,5,6,7,8,9])

# Logistic Regression

# Encoding

In [7]:
# encode the categorical varriable (encoding policy sales channel and Region Code makes high cardinality so i avoid encoding for this fields )
ohe = OneHotEncoder(sparse=False)        
transformed_train_data = ohe.fit_transform(train[["Age_Cat","Gender","Vehicle_Age","Previously_Insured","Driving_License","Vehicle_Damage"]])

# # the above transformed_data is an array so convert it to dataframe
encoded_train_data = pd.DataFrame(transformed_train_data, index=train.index)        
encoded_train_data.columns = ohe.get_feature_names(["Age_Cat","Gender","Vehicle_Age","Previously_Insured","Driving_License","Vehicle_Damage"])
train_data = pd.concat([train, encoded_train_data], axis=1)

# Scaling

In [8]:
sc =  StandardScaler()
vintage_scaled_array = sc.fit_transform(train_data[["Vintage","Region_Code","Policy_Sales_Channel","psc_success_rate","reg_success_rate"]])

minmaxsc = MinMaxScaler()
annual_premium_scaled_array = minmaxsc.fit_transform(train_data[["Annual_Premium"]])

train_data[["Vintage","Region_Code","Policy_Sales_Channel","psc_success_rate","reg_success_rate"]] = vintage_scaled_array
train_data["Sc_Annual_Premium"] = annual_premium_scaled_array

In [9]:
train_data.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,...,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Previously_Insured_0,Previously_Insured_1,Driving_License_0,Driving_License_1,Vehicle_Damage_No,Vehicle_Damage_Yes,Sc_Annual_Premium
0,1,Male,44,1,0.121784,0,> 2 Years,Yes,40454.0,-1.587234,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.070366
1,2,Male,76,1,-1.767879,0,1-2 Year,No,33536.0,-1.587234,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.057496
2,3,Male,47,1,0.121784,0,> 2 Years,Yes,38294.0,-1.587234,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.066347
3,4,Male,21,1,-1.163187,1,< 1 Year,No,28619.0,0.737321,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.048348
4,5,Female,29,1,1.104409,1,< 1 Year,No,27496.0,0.737321,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.046259


In [10]:
train_data.columns

Index(['id', 'Gender', 'Age', 'Driving_License', 'Region_Code',
       'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Annual_Premium',
       'Policy_Sales_Channel', 'Vintage', 'Response', 'psc_count',
       'psc_scount', 'psc_success_rate', 'reg_count', 'reg_scount',
       'reg_success_rate', 'Age_Cat', 'Age_Cat_1', 'Age_Cat_2', 'Age_Cat_3',
       'Age_Cat_4', 'Age_Cat_5', 'Age_Cat_6', 'Age_Cat_7', 'Age_Cat_8',
       'Gender_Female', 'Gender_Male', 'Vehicle_Age_1-2 Year',
       'Vehicle_Age_< 1 Year', 'Vehicle_Age_> 2 Years', 'Previously_Insured_0',
       'Previously_Insured_1', 'Driving_License_0', 'Driving_License_1',
       'Vehicle_Damage_No', 'Vehicle_Damage_Yes', 'Sc_Annual_Premium'],
      dtype='object')

In [11]:
trainX = train_data.drop(["id","Age","Age_Cat","Gender","Vehicle_Age","psc_count","psc_scount","reg_count","reg_scount","Vintage","Annual_Premium","Previously_Insured","Driving_License","Vehicle_Damage","Response"], axis=1)
trainY = train_data["Response"]

In [12]:
trainX.head()

,Region_Code,Policy_Sales_Channel,psc_success_rate,reg_success_rate,Age_Cat_1,Age_Cat_2,Age_Cat_3,Age_Cat_4,Age_Cat_5,Age_Cat_6,...,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Previously_Insured_0,Previously_Insured_1,Driving_License_0,Driving_License_1,Vehicle_Damage_No,Vehicle_Damage_Yes,Sc_Annual_Premium
0,0.121784,-1.587234,0.889268,1.426484,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.070366
1,-1.767879,-1.587234,0.889268,0.112584,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.057496
2,0.121784,-1.587234,0.889268,1.426484,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.066347
3,-1.163187,0.737321,-1.087420,-0.216476,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.048348
4,1.104409,0.737321,-1.087420,-0.017380,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.046259


## Modelling
Here target(Response) is a imbalanced data, So i have used stratified cross validation method


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,confusion_matrix
logreg = LogisticRegression(random_state=0,max_iter=1000)
models = {
    "Logistic Regression":logreg,
}
skf = StratifiedKFold(n_splits=5, shuffle=True)
for model_name,model in models.items():
    scores = []
    train_scores = []
    for train_indx,val_indx in skf.split(X=trainX,y=trainY):
        #get train and test data     
        X_train,X_val = trainX.loc[train_indx],trainX.loc[val_indx]
        Y_train,Y_val = trainY[train_indx],trainY[val_indx]
        model.fit(X_train,Y_train)
        #make a prediction
        Y_predict = model.predict_proba(X_val)

        accuracy = roc_auc_score(Y_val,Y_predict[:,1])
        Ytrain_predict = model.predict_proba(X_train)
        train_accuracy = roc_auc_score(Y_train,Ytrain_predict[:,1])        
        print("train",train_accuracy)
        print("test",accuracy)
        scores.append(accuracy)
        train_scores.append(train_accuracy)
    print("Mean Accurracy of test {0} is {1}".format(model_name,np.mean(scores)))
    print("Mean Accurracy of train {0} is {1}".format(model_name,np.mean(train_scores)))

train 0.8510331754446004
test 0.8486224308609367
train 0.84983563161646
test 0.8535350382309046
train 0.8506531891488874
test 0.8499643761991172
train 0.8508314329607071
test 0.8495893551681504
train 0.850570651386261
test 0.8505567994902197
Mean Accurracy of test Logistic Regression is 0.8504535999898657
Mean Accurracy of train Logistic Regression is 0.8505848161113831


# Random Forest Classifier

In [14]:
train

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,psc_count,psc_scount,psc_success_rate,reg_count,reg_scount,reg_success_rate,Age_Cat
0,1,Male,44,1,28,0,> 2 Years,Yes,40454.0,26,217,1,79700,15891.0,19.938519,106415,19917,18.716346,4
1,2,Male,76,1,3,0,1-2 Year,No,33536.0,26,183,0,79700,15891.0,19.938519,9251,1181,12.766187,7
2,3,Male,47,1,28,0,> 2 Years,Yes,38294.0,26,27,1,79700,15891.0,19.938519,106415,19917,18.716346,4
3,4,Male,21,1,11,1,< 1 Year,No,28619.0,152,203,0,134784,3858.0,2.862358,9232,1041,11.275997,2
4,5,Female,29,1,41,1,< 1 Year,No,27496.0,152,39,0,134784,3858.0,2.862358,18263,2224,12.177627,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381104,381105,Male,74,1,26,1,1-2 Year,No,30170.0,26,88,0,79700,15891.0,19.938519,2587,187,7.228450,7
381105,381106,Male,30,1,37,1,< 1 Year,No,40016.0,152,131,0,134784,3858.0,2.862358,5501,436,7.925832,2
381106,381107,Male,21,1,30,1,< 1 Year,No,35118.0,160,161,0,21779,475.0,2.181000,12191,900,7.382495,2
381107,381108,Female,68,1,14,0,> 2 Years,Yes,44617.0,124,74,0,73995,13996.0,18.914792,4678,422,9.020949,6


# Encoding(Ordinal Encoder)

In [15]:
oe = OrdinalEncoder()
train[["Gender","Vehicle_Damage","Vehicle_Age","Age_Cat"]] = oe.fit_transform(train[["Gender","Vehicle_Damage","Vehicle_Age","Age_Cat"]])

In [16]:
train

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,psc_count,psc_scount,psc_success_rate,reg_count,reg_scount,reg_success_rate,Age_Cat
0,1,1.0,44,1,28,0,2.0,1.0,40454.0,26,217,1,79700,15891.0,19.938519,106415,19917,18.716346,3.0
1,2,1.0,76,1,3,0,0.0,0.0,33536.0,26,183,0,79700,15891.0,19.938519,9251,1181,12.766187,6.0
2,3,1.0,47,1,28,0,2.0,1.0,38294.0,26,27,1,79700,15891.0,19.938519,106415,19917,18.716346,3.0
3,4,1.0,21,1,11,1,1.0,0.0,28619.0,152,203,0,134784,3858.0,2.862358,9232,1041,11.275997,1.0
4,5,0.0,29,1,41,1,1.0,0.0,27496.0,152,39,0,134784,3858.0,2.862358,18263,2224,12.177627,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381104,381105,1.0,74,1,26,1,0.0,0.0,30170.0,26,88,0,79700,15891.0,19.938519,2587,187,7.228450,6.0
381105,381106,1.0,30,1,37,1,1.0,0.0,40016.0,152,131,0,134784,3858.0,2.862358,5501,436,7.925832,1.0
381106,381107,1.0,21,1,30,1,1.0,0.0,35118.0,160,161,0,21779,475.0,2.181000,12191,900,7.382495,1.0
381107,381108,0.0,68,1,14,0,2.0,1.0,44617.0,124,74,0,73995,13996.0,18.914792,4678,422,9.020949,5.0


In [17]:
trainX = train.drop(["id","Age","Vintage","psc_count","psc_scount","reg_count","reg_scount","Response"], axis=1)
trainY = train["Response"]

In [18]:
trainX.dtypes

Gender                  float64
Driving_License           int64
Region_Code               int64
Previously_Insured        int64
Vehicle_Age             float64
Vehicle_Damage          float64
Annual_Premium          float64
Policy_Sales_Channel      int64
psc_success_rate        float64
reg_success_rate        float64
Age_Cat                 float64
dtype: object

In [19]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score,confusion_matrix
rf = RandomForestClassifier(n_estimators=7,max_depth=10)
models = {
    "rf":rf,
}
skf = StratifiedKFold(n_splits=5, shuffle=True)
for model_name,model in models.items():
    scores = []
    train_scores = []
    for train_indx,val_indx in skf.split(X=trainX,y=trainY):
        #get train and test data     
        X_train,X_val = trainX.loc[train_indx],trainX.loc[val_indx]
        Y_train,Y_val = trainY[train_indx],trainY[val_indx]
        model.fit(X_train,Y_train)
        #make a prediction
        Y_predict = model.predict_proba(X_val)

        accuracy = roc_auc_score(Y_val,Y_predict[:,1])
        Ytrain_predict = model.predict_proba(X_train)
        train_accuracy = roc_auc_score(Y_train,Ytrain_predict[:,1])        
        print("train",train_accuracy)
        print("test",accuracy)
        scores.append(accuracy)
        train_scores.append(train_accuracy)
    print("Mean Accurracy of test {0} is {1}".format(model_name,np.mean(scores)))
    print("Mean Accurracy of train {0} is {1}".format(model_name,np.mean(train_scores)))

train 0.8596493568222549
test 0.8506573729639976
train 0.8601931006268645
test 0.8523432090207932
train 0.8591510377849787
test 0.855661708320145
train 0.8609396986878122
test 0.8542996619232074
train 0.860032895437074
test 0.8548924413379865
Mean Accurracy of test rf is 0.8535708787132259
Mean Accurracy of train rf is 0.8599932178717967
